In [28]:
import pandas as pd
data = pd.read_csv("fashion_products.csv")
data.head()

,User ID,Product ID,Product Name,Brand,Category,Price,Rating,Color,Size
0,19,1,Dress,Adidas,Men's Fashion,40,1.043159,Black,XL
1,97,2,Shoes,H&M,Women's Fashion,82,4.026416,Black,L
2,25,3,Dress,Adidas,Women's Fashion,44,3.337938,Yellow,XL
3,57,4,Shoes,Zara,Men's Fashion,23,1.049523,White,S
4,79,5,T-shirt,Adidas,Men's Fashion,79,4.302773,Black,M


In [29]:
from surprise import Dataset, Reader, SVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

### First Approach: Content-Based Filtering


In [30]:
content_df = data[['Product ID', 'Product Name', 'Brand', 
                   'Category', 'Color', 'Size']]
content_df['Content'] = content_df.apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1)

# Use TF-IDF vectorizer to convert content into a matrix of TF-IDF features
tfidf_vectorizer = TfidfVectorizer()
content_matrix = tfidf_vectorizer.fit_transform(content_df['Content'])

content_similarity = linear_kernel(content_matrix, content_matrix)

reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(data[['User ID', 
                                  'Product ID', 
                                  'Rating']], reader)

def get_content_based_recommendations(product_id, top_n):
    index = content_df[content_df['Product ID'] == product_id].index[0]
    similarity_scores = content_similarity[index]
    similar_indices = similarity_scores.argsort()[::-1][1:top_n + 1]
    recommendations = content_df.loc[similar_indices, 'Product ID'].values
    return recommendations

<ipython-input-30-8b9183f797f5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  content_df['Content'] = content_df.apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1)


In [31]:
content_df.head(3)

,Product ID,Product Name,Brand,Category,Color,Size,Content
0,1,Dress,Adidas,Men's Fashion,Black,XL,1 Dress Adidas Men's Fashion Black XL
1,2,Shoes,H&M,Women's Fashion,Black,L,2 Shoes H&M Women's Fashion Black L
2,3,Dress,Adidas,Women's Fashion,Yellow,XL,3 Dress Adidas Women's Fashion Yellow XL


### Taking an example as: Recommendation for  purchasing Product  having productid = 4,


In [32]:
list_of_recommended_product_id = get_content_based_recommendations(4,5)

product_details = content_df[content_df['Product ID'].isin(list_of_recommended_product_id)]

product_details.reset_index(drop=True, inplace=True)

product_details


,Product ID,Product Name,Brand,Category,Color,Size,Content
0,195,Shoes,Zara,Kids' Fashion,White,L,195 Shoes Zara Kids' Fashion White L
1,236,Shoes,Zara,Men's Fashion,White,S,236 Shoes Zara Men's Fashion White S
2,668,Shoes,Zara,Men's Fashion,White,S,668 Shoes Zara Men's Fashion White S
3,714,Shoes,Zara,Men's Fashion,White,S,714 Shoes Zara Men's Fashion White S
4,908,Shoes,Zara,Kids' Fashion,White,M,908 Shoes Zara Kids' Fashion White M


### Second Approach: Collaborative Filtering


In [33]:
algo = SVD()
trainset = data.build_full_trainset()
algo.fit(trainset)

def get_collaborative_filtering_recommendations(user_id, top_n):
    testset = trainset.build_anti_testset()
    testset = filter(lambda x: x[0] == user_id, testset)
    predictions = algo.test(testset)
    predictions.sort(key=lambda x: x.est, reverse=True)
    recommendations = [prediction.iid for prediction in predictions[:top_n]]
    return recommendations

### Taking an example as: Recommendation for User  having userid = 1,

In [36]:
list_of_recommended_product_id= get_collaborative_filtering_recommendations(1,5)


product_details = content_df[content_df['Product ID'].isin(list_of_recommended_product_id)]

product_details.reset_index(drop=True, inplace=True)

product_details


,Product ID,Product Name,Brand,Category,Color,Size,Content
0,161,Sweater,Adidas,Women's Fashion,Blue,M,161 Sweater Adidas Women's Fashion Blue M
1,575,T-shirt,H&M,Kids' Fashion,White,L,575 T-shirt H&M Kids' Fashion White L
2,826,Shoes,Adidas,Women's Fashion,Red,S,826 Shoes Adidas Women's Fashion Red S
3,957,Jeans,Nike,Men's Fashion,White,L,957 Jeans Nike Men's Fashion White L
4,983,Dress,Zara,Men's Fashion,Red,XL,983 Dress Zara Men's Fashion Red XL


## And Finally, The Hybrid Approach
Now let’s combine content-based and collaborative filtering methods to build a recommendation system using the Hybrid method:

In [ ]:
def get_hybrid_recommendations(user_id, product_id, top_n):
    content_based_recommendations = get_content_based_recommendations(product_id, top_n)
    collaborative_filtering_recommendations = get_collaborative_filtering_recommendations(user_id, top_n)
    hybrid_recommendations = list(set(content_based_recommendations + collaborative_filtering_recommendations))
    return hybrid_recommendations[:top_n]

Here we are recommending products based on the product that a user is viewing:

In [ ]:
user_id = 6
product_id = 11
top_n = 10
recommendations = get_hybrid_recommendations(user_id, product_id, top_n)

print(f"Hybrid Recommendations for User {user_id} based on Product {product_id}:")
for i, recommendation in enumerate(recommendations):
    print(f"{i + 1}. Product ID: {recommendation}")
    print(f"{i + 1}. Product ID: {recommendation}")

Hybrid Recommendations for User 6 based on Product 11:
1. Product ID: 834
1. Product ID: 834
2. Product ID: 1123
2. Product ID: 1123
3. Product ID: 428
3. Product ID: 428
4. Product ID: 205
4. Product ID: 205
5. Product ID: 49
5. Product ID: 49
6. Product ID: 1234
6. Product ID: 1234
7. Product ID: 787
7. Product ID: 787
8. Product ID: 188
8. Product ID: 188
9. Product ID: 893
9. Product ID: 893
10. Product ID: 1790
10. Product ID: 1790
